# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。
    
    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。
    
    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。
    
    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

In [1]:
import os
import socket
import socks

socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 10808)
socket.socket = socks.socksocket
os.environ['SERPAPI_API_KEY'] = '9f8a1db8301e7dea21872755a392d2e65a82cd79'
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY'

In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9, max_tokens=500)

d:\Sdk\Anaconda3\envs\homework\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [4]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [5]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'product': "性能卓越的GPU"
    }))

d:\Sdk\Anaconda3\envs\homework\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(





1. 炫影科技有限责任公司（VisionTech Co., Ltd.）
2. 峰视图形处理器有限公司（Peak Vision Processor Co., Ltd.）
3. 灵光创新科技有限公司（Lightning Innovation Technology Co., Ltd.）
4. 强力视觉加速器有限责任公司（Powerful Vision Accelerator Co., Ltd.）
5. 超能图形芯片有限公司（Super Graphics Chip Co., Ltd.）
6. 幻影创意科技有限公司（Phantom Creative Technology Co., Ltd.）
7. 璀璨显卡有限责任公司（Brilliant Graphics Card Co., Ltd.）
8. 冲击力图形处理器有限公司（Impact Graphics Processor Co., Ltd.）
9. 极光视觉科技有限公司（Aurora Vision Technology Co., Ltd.）
10. 艺赛创新集成电路有限公司（Artis Innovation Integrated Circuit Co., Ltd.）


In [6]:
chain.verbose = True

In [7]:
chain.verbose

True

In [8]:
print(chain.run({
    'product': "性能卓越的GPU"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.


1. 流火图形技术有限公司(Lava Graphics Technology Co., Ltd.)
2. 极致视界芯片有限公司(Ultimate Vision Chip Co., Ltd.)
3. 强力绘图处理有限公司(Powerful Graphics Processing Co., Ltd.)
4. 飞跃数字渲染有限公司(Leap Digital Rendering Co., Ltd.)
5. 炫彩图像科技有限公司(Vivid Image Technology Co., Ltd.)
6. 火花计算设备有限公司(Spark Computing Devices Co., Ltd.)
7. 梦幻绘图科技有限公司(Fantasy Graphics Technology Co., Ltd.)
8. 极速视觉处理有限公司(Rapid Vision Processing Co., Ltd.)
9. 纵横图形芯片有限公司(Horizon Graphics Chip Co., Ltd.)
10. 强效图像加速器有限公司(Potent Image Accelerator Co., Ltd.)


In [9]:
print(chain.run({
    'product': "制作手办"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造制作手办的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.


1. 创意成形艺术手办有限公司(Creative Sculpture Art Co., Ltd.)
2. 动感造物手办有限公司(Dynamic Creations Co., Ltd.)
3. 立体绘制手办有限公司(3D Illustration Co., Ltd.)
4. 巧匠艺术手办有限公司(Skillful Art Co., Ltd.)
5. 秀色手办制作有限公司(Beautiful Figurines Production Co., Ltd.)
6. 灵动模型手办有限公司(Agile Models Co., Ltd.)
7. 精巧塑形手办有限公司(Exquisite Sculptures Co., Ltd.)
8. 炫彩动漫手办有限公司(Vibrant Anime Co., Ltd.)
9. 致美男女手办有限公司(Elegant Figures Co., Ltd.)
10. 玩转手办文化有限公司(Playful Figurines Culture Co., Ltd.)


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](../images/simple_sequential_chain_0.png)

In [10]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。

llm = OpenAI(temperature=0.7, max_tokens=1000)

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [11]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](../images/simple_sequential_chain_1.png)

In [12]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [13]:
review = overall_chain.run("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...


《三体人不是无法战胜的》是一部关于勇气、希望和挑战的戏剧。故事发生在一个虚构的宇宙中，那里有一种来自三个不同星球的生物，他们被称为三体人。这些生物拥有强大的科技能力，他们的目的是征服所有的星球，包括地球。在这个故事中，我们将跟随一群勇敢的地球人，他们决定不再被三体人压迫，而是挺身抵抗。在这场生死攸关的战斗中，地球人将面临前所未有的挑战，但他们也将发现自己拥有的力量和勇气。最终，地球人能否战胜三体人？他们的勇敢和决心能否改变宇宙的命运？这部戏剧将带领观众探索人类的内心世界，思考人类的本质和未来。它将给我们带来勇气和希望，让我们相信，三体人并不是无法战胜的。


《三体人不是无法战胜的》是一部充满勇气、希望和挑战的戏剧。它讲述了一群勇敢的地球人决定挺身抵抗来自三个不同星球的强大生物——三体人的故事。这部戏剧带领观众探索人类的内心世界，思考人类的本质和未来。

在这个虚构的宇宙中，三体人拥有强大的科技能力，他们的目的是征服所有的星球，包括地球。面对如此强大的对手，地球人面临前所未有的挑战，但他们决定不再被压迫，而是勇敢地反抗。这场生死攸关的战斗将考验地球人的勇气和决心，同时也让他们发现自己拥有的力量。

剧中的人物形象生动鲜明，每个人物都有自己的特点和内心世界。主人公们的勇气和决心将感染观众，让我们相信，即使面对强大的对手，人类也能战胜困难。同时，剧中也展现了人类的弱点和缺点，让我们反思人类的本质和未来。

这部戏剧充满了紧张刺激的战斗场面，同时也蕴含着深刻的思考和哲学意义。它不仅仅是一部娱乐作品，更是一部让观众思考人类命运和未来的作品。

总的来说，我强烈推荐观众去欣赏《三体人不是无法战胜的》，它将给你带来勇气和希望，让你相信，三体人并不是无法战胜的。它也将为观众带来一次深刻的思想之旅。

> Finished chain.


In [14]:
review = overall_chain.run("星球大战第九季")



> Entering new SimpleSequentialChain chain...


《星球大战第九季》是一部充满惊险、光明与黑暗的科幻戏剧。在遥远的宇宙中，帝国与反抗军之间的战争愈演愈烈。随着反抗军的力量日益增强，帝国的统治者决定发动最终的攻击，试图彻底摧毁反抗军。同时，反抗军领袖也不甘示弱，他们将展开一场决战，决定宇宙的命运。在这场战斗中，英雄与反派将迎来最终的决战，谁将成为宇宙的主宰？谁将获得胜利？谁又将付出巨大的代价？《星球大战第九季》将为观众带来一场震撼的冒险旅程，让我们一起见证最终的胜利者。


《星球大战第九季》展现了一场令人惊叹的决战，让观众们沉浸于宇宙的恢弘场景和精彩的战斗。这部戏剧不仅是一部科幻作品，更是一部关于光明与黑暗、英雄与反派之间永恒对抗的史诗般的故事。

导演和编剧们巧妙地将帝国与反抗军的战争打造成了一场激烈的斗争，让观众们紧张地跟随着每个角色的命运。反抗军领袖和帝国的统治者之间的决战更是让人屏息凝神，每个人都在猜测最终的胜利者会是谁。

除了令人惊叹的特效和动作场面外，这部戏剧也不乏深刻的情感和人物发展。每个角色都有自己的故事和动机，让观众们更加投入剧情。演员们的精湛演技也让这些角色栩栩如生，令人难忘。

总的来说，《星球大战第九季》是一部精彩绝伦的戏剧，它以惊险的决战为线索，将观众们带入一个充满想象力和戏剧性的宇宙。无论是科幻迷还是普通观众，都会被这部戏剧所吸引。不容错过！

> Finished chain.


### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](../images/sequential_chain_0.png)

In [15]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
llm = OpenAI(temperature=.7, max_tokens=1000)
template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [16]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [17]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [18]:
m_overall_chain({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})

d:\Sdk\Anaconda3\envs\homework\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：


在二十一世纪的新中国，人类社会已经迈入了一个新的时代，科技发展迅速，生活水平不断提高。然而，与此同时，人类也面临着前所未有的挑战——来自外太空的三体人入侵。这群来自三体星系的外星人拥有强大的科技力量，他们的目的是摧毁地球文明，将地球变成他们的新家园。

在这个危机四伏的时代，一群勇敢的中国人挺身而出，组成了抵抗三体人入侵的队伍。他们经历了无数的艰难困苦，但始终不放弃，用自己的智慧和勇气为人类的未来而战。在与三体人的殊死搏斗中，他们发现三体人并非无法战胜，只是需要找到他们的弱点并加以利用。

在这场生死搏斗中，人类面临着最艰难的选择，也展现出最伟大的勇气和智慧。最终，人类战胜了三体人，守护了地球的和平，也为自己的未来打下了坚实的基础。这部戏剧讲述了人类面对危机时的顽强和勇敢，以及科技与智慧的力量。它也提醒我们，虽然生活充满挑战，但只要团结一心，就没有什么是无法战胜的。

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '\n\n在二十一世纪的新中国，人类社会已经迈入了一个新的时代，科技发展迅速，生活水平不断提高。然而，与此同时，人类也面临着前所未有的挑战——来自外太空的三体人入侵。这群来自三体星系的外星人拥有强大的科技力量，他们的目的是摧毁地球文明，将地球变成他们的新家园。\n\n在这个危机四伏的时代，一群勇敢的中国人挺身而出，组成了抵抗三体人入侵的队伍。他们经历了无数的艰难困苦，但始终不放弃，用自己的智慧和勇气为人类的未来而战。在与三体人的殊死搏斗中，他们发现三体人并非无法战胜，只是需要找到他们的弱点并加以利用。\n\n在这场生死搏斗中，人类面临着最艰难的选择，也展现出最伟大的勇气和智慧。最终，人类战胜了三体人，守护了地球的和平，也为自己的未来打下了坚实的基础。这部戏剧讲述了人类面对危机时的顽强和勇敢，以及科技与智慧的力量。它也提醒我们，虽然生活充满挑战，但只要团结一心，就没有什么是无法战胜的。',
 'review': '\n\n《三体》是一部充满惊险与希望的戏剧作品。它不仅展现了现代中国的科技发展和生活水平提高，更重要的是，它通过对抗外星入侵的故事，呈现了人类面临危机时的勇气和智慧。\n\n该剧的故事情节紧凑，充满惊险和悬念。观众将被带入一个充满未知和挑战的世界，与主人公一起经历各种艰难困苦。同时，该剧也展现了中国人民的团结和勇气，以及科技与智慧的力量。观众将在剧中感受到一股强大的希望力量，让他们相信只要团结一心，就没有什么是无法战胜的。\n\n此外，该剧的舞台设计和服装造型也令人印象深刻。通过细腻的舞台布置和精美的服装，观众可以更加直观地感受到剧中的未来世界和外星人的科技力量。\n\n总的来说，《三体》是一部值得观看的戏剧作品。它不仅展现了中国人民的团结与勇气，更重要的是，它提醒我们，面对任何挑战，只要拥有智慧和勇气，就能够战胜一切。这部戏剧将带给观众希望和力量，让他们相信未来充满无限可能。'}

### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果